In [ ]:
!pip install transformers datasets

In [ ]:
import re
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

# Procesamiento del chat de WhatsApp para la creación del dataset

Primero debemos cargar un archivo de chat de WhatsApp (llamado chat.txt por defecto). Luego filtramos mensajes irrelevantes, como por ejemplo aquellos donde se encontraban mensajes multimedia, y separamos las conversaciones en prompts y responses. Además capturamos los mensajes del autor objetivo ("Nico Bazan" en este caso) y los emparejamos con los mensajes de los otros participantes del chat sin importar sus nombres para usarlos como entrenamiento.
Es importante notar que también se verifica que los mensajes no estén vacíos.

In [ ]:
# Datos irrelevantes a ignorar
irrelevantData = {
    'eliminaste este mensaje.',
    'se eliminó este mensaje.',
    '<multimedia omitido>',
    'Los mensajes y las llamadas están cifrados de extremo a extremo. nadie fuera de este chat, ni siquiera whatsApp, puede leerlos ni escucharlos. toca para obtener más información.',
    'you deleted this message',
    'this message was deleted',
    '<media omitted>',
    'messages and calls are end-to-end encrypted. no one outside of this chat, not even whatsApp, can read or listen to them. tap to learn more.',

}

# Preprocesar el texto: convertir a minúsculas y eliminar stopwords
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower().strip()

    return text

# Verificar si el mensaje es irrelevante
def containsIrrelevantData(message):
    return any(irrelevant in message for irrelevant in irrelevantData)

# Procesar el chat con múltiples autores
def process_whatsapp_chat(filepath, target_author):
    with open(filepath, 'r', encoding='utf-8') as file:
        chat_lines = file.readlines()

    prompts = []
    responses = []

    current_prompt = ""
    current_response = ""
    in_prompt = True  # Indica si estamos acumulando en el prompt

    for line in chat_lines:
        # Extraer fecha, autor y contenido del mensaje
        match = re.match(r'\d+/\d+/\d+,\s\d+:\d+\s-\s([^:]+):\s(.+)', line)

        if match:
            author = match.group(1).strip()
            message = preprocess_text(match.group(2).strip())

            # Ignorar mensajes irrelevantes
            if containsIrrelevantData(message):
                continue

            if author != target_author:  # Otros autores
                if in_prompt:
                    current_prompt += f"{message}\n"  # Agregar con salto de línea
                else:
                    # Guardar el par si no está vacío
                    if current_prompt.strip() and current_response.strip():
                        prompts.append(current_prompt.strip())
                        responses.append(current_response.strip())
                    # Reiniciar para el siguiente ciclo
                    current_prompt = f"{message}\n"
                    current_response = ""
                    in_prompt = True  # Volver a modo prompt
            else:  # Mensajes del autor objetivo (Nico Bazan)
                if not in_prompt:
                    current_response += f"{message}\n"  # Agregar con salto de línea
                else:
                    in_prompt = False  # Cambiar a modo respuesta
                    current_response = f"{message}\n"

    # Guardar el último par si no está vacío
    if current_prompt.strip() and current_response.strip():
        prompts.append(current_prompt.strip())
        responses.append(current_response.strip())

    return prompts, responses

# Procesar el chat
chat_file = 'chat.txt'
target_author = '' # Reemplazar con tu propio nombre
prompts, responses = process_whatsapp_chat(chat_file, target_author)

# Eliminar pares vacíos
prompts, responses = zip(*[
    (p, r) for p, r in zip(prompts, responses) if p.strip() and r.strip()
])

In [ ]:
# Imprimir 3 ejemplos limpios de prompts y responses
for i in range(min(3, len(prompts))):
    print(f"Prompt {i+1}: {prompts[i]}")
    print(f"Response {i+1}: {responses[i]}")
    print("-" * 40)

# Shuffle de datos

In [ ]:
import random

# Crear una lista de pares (prompt, response)
examples = list(zip(prompts, responses))

# Mezclar los pares aleatoriamente
random.shuffle(examples)

# Separar nuevamente los prompts y responses mezclados
prompts, responses = zip(*examples)

# Mostrar algunos ejemplos aleatorios (opcional)
for i, (p, r) in enumerate(zip(prompts[:10], responses[:10]), start=1):
    print(f"Ejemplo {i}:\nPrompt: {p}\nResponse: {r}\n")


## Creación del archivo de entrenamiento con delimitadores

Convertiremos los prompts y responses procesados en un formato estructurado utilizando delimitadores claros (PROMPT, RESPONSE y ###). Cada par de conversación se guarda en el archivo train_data.txt para su posterior uso en el entrenamiento del modelo.

In [ ]:
import pandas as pd
import csv

# Crear un DataFrame con las columnas 'prompt' y 'response'
data = pd.DataFrame({
    "prompt": prompts,
    "response": responses
})

# Guardar el DataFrame en un archivo CSV con delimitadores claros y manteniendo los saltos de línea
data.to_csv('train_data.csv', index=False, encoding='utf-8', lineterminator='\n', quoting=csv.QUOTE_ALL)

print("Datos guardados correctamente en 'train_data.csv'.")


## Filtrar mensajes descontextualizados


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Cargar el CSV
data = pd.read_csv('train_data.csv')

# Filtrar mensajes muy cortos o sin contenido significativo
data = data[(data['prompt'].str.split().str.len() > 2) & (data['response'].str.split().str.len() > 2)]

# Filtrar mensajes donde prompt o response son exactamente iguales a palabras irrelevantes
irrelevant_words = {}
# irrelevant_words = {'xd', ':cc', ':(', 'ok'}
data = data[~data['prompt'].isin(irrelevant_words) & ~data['response'].isin(irrelevant_words)]

# Calcular embeddings de cada prompt y response
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
embeddings_prompt = model.encode(data['prompt'].tolist(), convert_to_tensor=True)
embeddings_response = model.encode(data['response'].tolist(), convert_to_tensor=True)

# Mover los embeddings a la CPU antes de calcular similitud
embeddings_prompt = embeddings_prompt.cpu()
embeddings_response = embeddings_response.cpu()

# Calcular similitud entre cada prompt y response
similarities = util.pytorch_cos_sim(embeddings_prompt, embeddings_response).diagonal()

# Convertir las similitudes a un tensor de numpy y luego a una lista
similarities = similarities.numpy()

# Agregar la columna de similitud al DataFrame
data['similarity'] = similarities

# Definir umbral de similitud
similarity_threshold = 0.2

# Filtrar ejemplos con alta similitud para el entrenamiento y baja similitud para revisión
data_high_similarity = data[data['similarity'] > similarity_threshold]
data_low_similarity = data[data['similarity'] <= similarity_threshold]

# Guardar ambos conjuntos en archivos CSV separados
data_high_similarity = data_high_similarity.drop(columns=['similarity'])
data_low_similarity.to_csv('low_similarity_data.csv', index=False, encoding='utf-8')  # Mantiene la columna de 'similarity'

data_high_similarity.to_csv('filtered_train_data.csv', index=False, encoding='utf-8')

print("Datos filtrados guardados en 'filtered_train_data.csv'.")
print("Pares con baja similitud guardados en 'low_similarity_data.csv' con el nivel de similitud.")


## Carga y configuración del modelo GPT-2 en español

Ahora realizamos la configuración del modelo GPT-2 en español que será fine tuneado utilizando la librería transformers. Además, se ajustan parámetros como:

* Carga del modelo base y el tokenizador GPT-2 con un model_max_length de 64.
* Configuración del dropout (comentada inicialmente, pero se puede ajustar para el entrenamiento).
* Alineación del pad_token con `eos_token,** garantizando que el padding no interfiera con el procesamiento del modelo.
* Ajuste de embeddings: Se asegura de que la dimensión del vocabulario se adapte al tokenizador cargado.
* Sincronización del peso de la capa de salida (lm_head) con los embeddings iniciales para evitar inconsistencias.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo base en español
model_name = "DeepESP/gpt2-spanish"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="right")

# Configurar dropout (esto se aplicará durante el entrenamiento)
model.config.attn_pdrop = 0.3
model.config.embd_pdrop = 0.3
model.config.resid_pdrop = 0.3


# Alinear el token de padding con el token de fin de secuencia
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))  # Ajustar embeddings

# Inicializa el peso de la lm_head si es necesario
with torch.no_grad():
    model.lm_head.weight = model.transformer.wte.weight

## Carga y creación del dataset para entrenamiento y validación

Esta celda carga los datos desde el archivo train_data.txt, los procesa utilizando delimitadores claros y los convierte en un dataset compatible con Hugging Face. A continuación, el dataset se divide en subconjuntos para entrenamiento y validación (80-20), garantizando que los datos se mezclen aleatoriamente.

In [ ]:
from datasets import load_dataset, DatasetDict

# Cargar el dataset directamente desde el archivo CSV
dataset = load_dataset('csv', data_files='filtered_train_data.csv')

# Dividir el dataset en entrenamiento y validación
train_test_split = dataset['train'].train_test_split(test_size=0.2, shuffle=True)

# Crear un DatasetDict para entrenamiento y validación
datasets = DatasetDict({
    "train": train_test_split["train"],
    "validation": train_test_split["test"]
})

# prompts y responses limpias
dataset_prompt = dataset["train"]["prompt"]
dataset_response = dataset["train"]["response"]

# Verificar la estructura del dataset
print("Primeros ejemplos de entrenamiento:\n", datasets["train"][:2])



## Definición de funciones auxiliares para tokenización y agrupación de textos

Esta celda define dos funciones auxiliares esenciales para la preparación del dataset y el entrenamiento del modelo. Estas funciones se encargan de tokenizar los textos y agruparlos en bloques del tamaño adecuado para optimizar el entrenamiento del modelo.
Devuelven un nuevo dataset en el que los textos están tokenizados (convertidos en índices del vocabulario) y se incluye una máscara de atención para distinguir los tokens relevantes de los tokens de padding. Además, se preparan labels para el entrenamiento.

(Créditos a [Cristian Cardellino](https://crscardellino.net) por estas [funciones](https://huggingface.co/crscardellino/flisol-cba-martin-fierro/resolve/main/utils.py))

In [ ]:
from datasets import DatasetDict
from transformers import PreTrainedTokenizerBase
from typing import Callable, Dict, List

def tokenize(
    tokenizer: PreTrainedTokenizerBase
) -> Callable[[Dict[str, List[str]]], DatasetDict]:
    """
    Tokeniza los textos combinando prompt y response sin delimitadores.

    Parameters
    ----------
    tokenizer : PreTrainedTokenizerBase
        El tokenizador a usar para el proceso de tokenización.

    Returns
    -------
    Callable[[Dict[str, List[str]]], DatasetDict]
        La función encargada del proceso de tokenización.
    """
    def _tokenize(examples: Dict[str, List[str]]) -> DatasetDict:
        # Concatenar prompt y response sin delimitadores
        texts = [f"{p} {r}".strip() for p, r in zip(examples["prompt"], examples["response"])]
        return tokenizer(texts, truncation=True, padding="max_length", max_length=128)

    return _tokenize


def group_texts(examples: Dict[str, List[int]], block_size: int = 128) -> Dict[str, List[int]]:
    """
    Agrupa textos en bloques del tamaño especificado para asegurar longitud uniforme en el entrenamiento.

    Parameters
    ----------
    examples : Dict[str, List[int]]
        Diccionario con los datos tokenizados, contiene 'input_ids' y 'attention_mask'.
    block_size : int
        Tamaño de bloque para dividir los textos en el proceso de entrenamiento.

    Returns
    -------
    Dict[str, List[int]]
        El diccionario que contiene el nuevo dataset dividido en bloques de `block_size`.
    """
    # Concatenar todos los textos
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples["input_ids"])

    # Ajustar el total_length para evitar remainders incompletos
    total_length = (total_length // block_size) * block_size

    # Dividir en bloques del tamaño especificado
    result = {
        k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }

    # Copiar `input_ids` como `labels` para el entrenamiento
    result["labels"] = result["input_ids"].copy()

    return result


## Tokenización del dataset para preparación del entrenamiento

En esta celda se realiza la tokenización de los datos de entrenamiento y validación, preparando los textos para que puedan ser procesados por el modelo. La función de tokenización transforma los textos en índices del vocabulario y añade padding para mantener la consistencia en la longitud de los ejemplos.

In [ ]:
from functools import partial

def tokenize_function(examples):
    """
    Combina el prompt y response de cada entrada sin delimitadores para tokenizar.
    """
    # Concatenar prompt y response sin los delimitadores
    texts = [f"{p} {r}".strip() for p, r in zip(examples["prompt"], examples["response"])]
    return tokenizer(
        texts,
        truncation=True,
        max_length=128,
        padding="max_length"
    )

# Usar partial para fijar el tokenizador en la función
tokenize_with_tokenizer = partial(tokenize_function)

# Aplicar la tokenización al dataset usando map
tokenized_datasets = datasets.map(
    tokenize_with_tokenizer,
    batched=True,
    num_proc=2,  # Reducido para evitar problemas de memoria
    remove_columns=["prompt", "response"]
)

# Verificar los primeros datos tokenizados
print("Datos tokenizados:\n", tokenized_datasets["train"][:2])


## Agrupación de textos en bloques del tamaño especificado

Se aplica la función group_texts al dataset previamente tokenizado para dividir los tokens en bloques del tamaño especificado (por defecto, 128 tokens). Esto asegura que los lotes (batches) tengan una longitud uniforme durante el entrenamiento.

In [ ]:
# Agrupar los textos en bloques
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=2
)

# Verificar los primeros ejemplos agrupados (solo los primeros 20 tokens por campo)
print("Datos agrupados (primeros 20 tokens):")
for i in range(2):
    print(f"Ejemplo {i+1}:")
    for k, v in lm_datasets["train"][i].items():
        print(f"  {k}: {v[:20]}")


## Configuración del entrenamiento y optimización del modelo

Se configura los parámetros clave para el entrenamiento del modelo mediante el uso de TrainingArguments y el objeto Trainer. Se define el optimizador AdamW con un scheduler cosine para ajustar dinámicamente la tasa de aprendizaje. Además, se calcula el número de pasos totales y de calentamiento (warmup) para controlar la tasa al inicio del entrenamiento. La configuración incluye la opción de EarlyStopping para detener el entrenamiento si no hay mejora en la pérdida, optimizando así los recursos.

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Configuración de los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./fine_tuned_gpt2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,   # Ajustado si hay problemas de memoria
    learning_rate=2e-6,
    weight_decay=0.01,
    logging_steps=5,
    report_to=[],
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False
)

# Configuración del Trainer con Early Stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Ajuste opcional de paciencia
)


## Inicio del Entrenamiento del Modelo
Ejecuta el proceso de fine-tuning

In [ ]:
trainer.train()

## Guardar el modelo fine-tuneado

Una vez finalizado el entrenamiento, guarda el modelo fine-tuneado y su tokenizador para reutilizarlos más adelante.

In [ ]:
model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')

## Generación y Comparación de Respuestas entre el Modelo Base y el Modelo Fine-Tuneado

Esta celda implementa una función para generar respuestas del modelo y comparar los resultados entre el modelo base y el fine-tuneado. Se utiliza la misma entrada (prompt) en ambos modelos para observar cómo el fine tunning mejora (o cambia) las respuestas.

In [ ]:
# Función para generar una respuesta
def generate_response(prompt, model, tokenizer, max_length=50):
    model.eval()  # Asegurarse de que el modelo esté en modo evaluación
    inputs = tokenizer(prompt, return_tensors='pt', padding=True).to(device)

    # Generar salida del modelo
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=50,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id,
        top_p=0.95,  # Controlar diversidad
        top_k=50,   # Opciones más probables
        temperature=0.8,  # Controlar aleatoriedad
        do_sample=True  # Habilitar muestreo aleatorio
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):].strip()

# Cargar ambos modelos: base y fine-tuneado
base_model = AutoModelForCausalLM.from_pretrained("DeepESP/gpt2-spanish").to(device)
fine_tuned_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_gpt2").to(device)

# Comparar respuestas entre ambos modelos
def compare_models(prompt):
    print(f"Prompt: {prompt}")

    # Generar respuesta con el modelo base
    base_response = generate_response(prompt, base_model, tokenizer)
    print(f"Respuesta del Modelo Base:\n{base_response}\n")

    # Generar respuesta con el modelo fine-tuneado
    fine_tuned_response = generate_response(prompt, fine_tuned_model, tokenizer)
    print(f"Respuesta del Modelo Fine-Tuneado:\n{fine_tuned_response}")

    print("-" * 40)

# Comparar respuestas usando prompts limpios del dataset
for i in range(2):
    randint = random.randint(0, len(dataset["train"]["prompt"]))
    prompt = dataset["train"]["prompt"][randint]
    compare_models(prompt)

In [ ]:
import random
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModelForCausalLM, AutoTokenizer

# Cargar el modelo fine-tuneado y el tokenizador
fine_tuned_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_gpt2").to(device)
tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Función para obtener un prompt aleatorio
def get_random_prompt(prompts):
    if not prompts:
        return None
    return random.choice(prompts)

# Generar respuesta para un prompt aleatorio y calcular similitud
def generate_response_and_calculate_similarity(prompt,ft):
    if ft:
      # Generar respuesta del modelo fine-tuneado
      response = generate_response(prompt, fine_tuned_model, tokenizer)
    else:
      # Generar respuesta del modelo *no* fine-tuneado
      response = generate_response(prompt, base_model, tokenizer)

    # Calcular embeddings para prompt y respuesta generada
    prompt_embedding = embedding_model.encode(prompt, convert_to_tensor=True).cpu()
    response_embedding = embedding_model.encode(response, convert_to_tensor=True).cpu()

    # Calcular similitud entre el prompt y la respuesta generada
    similarity = util.pytorch_cos_sim(prompt_embedding, response_embedding).item()

    return similarity, response


# Obtener un prompt aleatorio y calcular la similitud con la respuesta generada
prompt = get_random_prompt(dataset_prompt)
sim, resp = generate_response_and_calculate_similarity(prompt,True)

# Mostrar el prompt, la respuesta generada y la similitud
print(f"Prompt: {prompt}\n")
print(f"Respuesta del Modelo Fine-Tuneado:\n{resp}\n")
print(f"Similitud: {sim:.4f}")


## Comparación de similitud semántica del modelo Fine Tuneado y el modelo base

In [ ]:
import math
comparation_info_ft = []
comparation_info_not_ft = []

for i in range(0, math.floor(len(dataset_prompt)/100)):
  aux_prompt = dataset_prompt[i]
  aux_similarity_ft, res_ft = generate_response_and_calculate_similarity(aux_prompt,True)
  aux_similarity_not_ft, res_not_ft = generate_response_and_calculate_similarity(aux_prompt,False)
  comparation_info_ft.append(aux_similarity_ft)
  comparation_info_not_ft.append(aux_similarity_not_ft)

Gráfico comparativo:

In [ ]:
import matplotlib.pyplot as plt

def group_similarities(info):
  # Inicializar una lista para contar la cantidad en cada categoría
  # Similitud entre 0 y 0.1, 0.1 y 0.2, ..., entre 0.6 y 1
  categories = [0] * 7

  for elem in info:
    if elem < 0.1:
      categories[0] += 1
    elif 0.1 <= elem < 0.2:
      categories[1] += 1
    elif 0.2 <= elem < 0.3:
      categories[2] += 1
    elif 0.3 <= elem < 0.4:
      categories[3] += 1
    elif 0.4 <= elem < 0.5:
      categories[4] += 1
    elif 0.5 <= elem < 0.6:
      categories[5] += 1
    elif 0.6 <= elem <= 1:
      categories[6] += 1

  return categories

# Generación de gráficos barra para comparar la similitud semántica entre el modelo fine-tuneado y el modelo base
grouped_similarities_ft = group_similarities(comparation_info_ft)
grouped_similarities_not_ft = group_similarities(comparation_info_not_ft)
categorias = ['<0.1', '<0.2', '<0.3', '<0.4', '<0.5', '<0.6', '<1']
max_y = max(max(grouped_similarities_ft), max(grouped_similarities_not_ft))

plt.bar(categorias, grouped_similarities_ft, color='#8b9684')
plt.title('Similitud del modelo Fine-Tuneado')
plt.xlabel('Similitud menor a')
plt.ylabel('Cantidad de respuestas con esa similitud')
plt.ylim(0, max_y)

plt.show()

plt.bar(categorias, grouped_similarities_not_ft, color='#8b9684')
plt.title('Similitud del modelo base')
plt.xlabel('Similitud menor a')
plt.ylabel('Cantidad de respuestas con esa similitud')
plt.ylim(0, max_y)

plt.show()


## Chat Interactivo con el Modelo Fine-Tuneado

 Acá se implementa un chat interactivo con el modelo fine-tuneado, permitiendonos conversar directamente con el chat bot. El ciclo permanece activo hasta que escribamos 'terminar', momento en el que la conversación finaliza.

In [ ]:
print("Bienvenido al chat con el bot. Escribe 'terminar' para finalizar la conversación.")

while True:
    user_input = input("Vos: ")  # Capturar entrada del usuario
    if user_input.lower() == "terminar":
        print("Bot: Hasta luego!")
        break  # Finaliza el chat si el usuario escribe "terminar"

    # Generar la respuesta del bot con longitud máxima de 50 tokens
    response = generate_response(user_input, fine_tuned_model, tokenizer, max_length=50)
    print(f"Bot: {response}")